In [2]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='10abcd80153e480aa1309b5782db7a0a', client_secret='c6c037c908344f439fe469dcfbfb8918'))

In [3]:
username = 'mythreya75'
playlist = 'spotify:playlist:1iVtAjR9ZFFkTcAoOlDqpf'

# track_features[track_name] = track_uri
track_features = {}


playlist = spotify.user_playlist_tracks(username, playlist, fields=None, limit=100, offset=0, market=None)


for track in playlist['items']:
    ident = track['track']['id']
    feature_data = list(spotify.audio_features(ident)[0].values())
    # print(spotify.audio_features(ids)[0])
    track_features[track['track']['name']] = feature_data

    
pd.set_option('display.max_rows', 500)

# print(track_features)
df = pd.DataFrame.from_dict(track_features, orient="index", columns=['danceability', 'energy','key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Guatemala - From Swaecation,0.849,0.699,4,-5.193,0,0.0574,0.185000,0.001180,0.0954,0.0778,100.989,audio_features,0TCnOEVeLQMXOUrpPlM7uY,spotify:track:0TCnOEVeLQMXOUrpPlM7uY,https://api.spotify.com/v1/tracks/0TCnOEVeLQMX...,https://api.spotify.com/v1/audio-analysis/0TCn...,256341,4
Unforgettable,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.7330,97.985,audio_features,3B54sVLJ402zGa6Xm4YGNe,spotify:track:3B54sVLJ402zGa6Xm4YGNe,https://api.spotify.com/v1/tracks/3B54sVLJ402z...,https://api.spotify.com/v1/audio-analysis/3B54...,233902,4
30 Hours,0.823,0.813,7,-6.710,1,0.2880,0.035900,0.000000,0.0886,0.3730,95.012,audio_features,2CHmgtK8OCL28WtIK96u4N,spotify:track:2CHmgtK8OCL28WtIK96u4N,https://api.spotify.com/v1/tracks/2CHmgtK8OCL2...,https://api.spotify.com/v1/audio-analysis/2CHm...,323303,4
No More Parties In LA,0.508,0.921,5,-1.644,1,0.3030,0.246000,0.000000,0.2210,0.6810,93.058,audio_features,0zLClc0emc6qUeV1p5nc99,spotify:track:0zLClc0emc6qUeV1p5nc99,https://api.spotify.com/v1/tracks/0zLClc0emc6q...,https://api.spotify.com/v1/audio-analysis/0zLC...,374413,4
Feedback,0.811,0.566,8,-6.033,0,0.5170,0.563000,0.000000,0.1040,0.1690,100.224,audio_features,49fT6owWuknekShh9utsjv,spotify:track:49fT6owWuknekShh9utsjv,https://api.spotify.com/v1/tracks/49fT6owWukne...,https://api.spotify.com/v1/audio-analysis/49fT...,147240,4
Bam,0.639,0.788,6,-2.879,1,0.3500,0.455000,0.000000,0.0580,0.8470,155.913,audio_features,6Iuh3vNjpWhSeHM3RZgzQL,spotify:track:6Iuh3vNjpWhSeHM3RZgzQL,https://api.spotify.com/v1/tracks/6Iuh3vNjpWhS...,https://api.spotify.com/v1/audio-analysis/6Iuh...,234973,4
Taste (feat. Offset),0.866,0.613,0,-6.385,1,0.1510,0.029300,0.000000,0.1040,0.3590,98.001,audio_features,2toVe5hfuIi97ytDPDbQFt,spotify:track:2toVe5hfuIi97ytDPDbQFt,https://api.spotify.com/v1/tracks/2toVe5hfuIi9...,https://api.spotify.com/v1/audio-analysis/2toV...,229210,4
Ric Flair Drip (with Metro Boomin),0.880,0.428,9,-8.280,1,0.2060,0.149000,0.000051,0.1140,0.3330,100.007,audio_features,7sO5G9EABYOXQKNPNiE9NR,spotify:track:7sO5G9EABYOXQKNPNiE9NR,https://api.spotify.com/v1/tracks/7sO5G9EABYOX...,https://api.spotify.com/v1/audio-analysis/7sO5...,172800,4
goosebumps,0.841,0.728,7,-3.370,1,0.0484,0.084700,0.000000,0.1490,0.4300,130.049,audio_features,6gBFPUFcJLzWGx4lenP6h2,spotify:track:6gBFPUFcJLzWGx4lenP6h2,https://api.spotify.com/v1/tracks/6gBFPUFcJLzW...,https://api.spotify.com/v1/audio-analysis/6gBF...,243837,4
IV. Sweatpants,0.710,0.470,1,-9.330,0,0.1420,0.154000,0.000000,0.6430,0.5810,80.027,audio_features,00GOPLxW4PGQuUYdPJh8K1,spotify:track:00GOPLxW4PGQuUYdPJh8K1,https://api.spotify.com/v1/tracks/00GOPLxW4PGQ...,https://api.spotify.com/v1/audio-analysis/00GO...,180675,4
